## Compressed sensing with QFT

**Compressed sensing objective**

Given a high dimensional signal $\mathbf{x} \in\mathbb{R}^{D}$ we want to find a vector $\mathbf{s}^*$ in the set. 

$$
\argmin_{\mathbf{s} \in \mathbb{R}^{D}}  \|\mathbf{s}\|_1  \\
\text{s.t. } C\mathbf{x} = CF^{-1}\mathbf{s}
$$


**Tensorized extension of compressed sensing**

Given a high dimensional signal $\mathcal{x} \in\mathbb{R}^{D\times\cdots\times D}$ represneted in TT format with ranks $\{R_i\}_{i=1}^N$ we want to find a TT-vector $\mathbf{s}^*$ in the set. 

$$
\argmin_{\mathcal{s} \in \mathbb{R}^{D}}  \|\mathcal{s}\|_1  \\
\text{s.t. } \mathcal{C}\mathcal{x} = \mathcal{C}\mathcal{F}^{-1}\mathcal{s}
$$

where $\mathcal{C}\,\&\,\mathcal{F}$ are Matrix Product Operators (MPO).


In [12]:
import torch
import tntorch as tn

# Hyperparameters
L, R, D = 8, 8, 2

dims = (D,)*L
s = tn.randn(*dims, ranks_tt=R, requires_grad=True)

In [6]:
# !pip install quimb
import quimb as qu
import quimb.tensor as qtn

In [ ]:
s_mps = qtn.MPS_rand_state(L=L, bond_dim=R, tags=['S'])  # sparse tensor

# This will be the combined inverse fourier MPO + sampler MPO
w_mpo = qtn.MPO_rand_herm(L=L, bond_dim=R, tags=['HAM'])  

# Target we want to reconstruct
x_targ_mps = qtn.MPS_rand_state(L=L, bond_dim=R, tags=['X'])  # dense tensor

print("s")
print(s_mps.show())

print("\nw_mpo")
print(w_mpo.show())

s
 8 8 8 8 8 8 8 
●─●─●─●─●─●─●─●
│ │ │ │ │ │ │ │
None

w_mpo
│8│8│8│8│8│8│8│
●─●─●─●─●─●─●─●
│ │ │ │ │ │ │ │
None


In [44]:
def loss_fn(s_mps, w_mpo, x_targ_mps):
    x_hat_mps = w_mpo.apply(other=s_mps)
    psi = x_hat_mps - x_targ_mps
    return psi.norm()

loss_fn(s_mps, w_mpo, x_targ_mps)

1.001614892659119

In [47]:
tnopt = qtn.TNOptimizer(
    # the tensor network we want to optimize
    s_mps,
    # the functions specfying the loss and normalization
    loss_fn=loss_fn,
    # norm_fn=norm_fn,
    # we specify constants so that the arguments can be converted
    # to the  desired autodiff backend automatically
    loss_constants={"w_mpo": w_mpo, "x_targ_mps": x_targ_mps},
    # the underlying algorithm to use for the optimization
    # 'l-bfgs-b' is the default and often good for fast initial progress
    optimizer="adam",
    # which gradient computation backend to use
    autodiff_backend="torch",
)
tnopt

<TNOptimizer(d=800, backend=torch)>

In [48]:
s_mps_opt = tnopt.optimize(1000)

+0.305455653889 [best: +0.305455653889] : : 1001it [00:03, 266.70it/s]                        
